# Tree Mapping with SAMGeo and Segment Anything Model 2 (SAM 2)

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/tree_mapping.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/tree_mapping.ipynb)

This notebook shows how to segment trees from aerial imagery with the Segment Anything Model 2 (SAM 2).

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [14]:
%pip install segment-geospatial[samgeo2] leafmap localtileserver

In [15]:
import leafmap
from samgeo import SamGeo2

## Create an interactive map

In [16]:
m = leafmap.Map(center=[-22.17615, -51.253043], zoom=18, height="800px")
m.add_basemap("SATELLITE")
m

Map(center=[-22.17615, -51.253043], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [17]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-51.2565, -22.1777, -51.2512, -22.175]

In [18]:
image = "Image.tif"
leafmap.map_tiles_to_geotiff(
    output=image, bbox=bbox, zoom=19, source="Satellite", overwrite=True
)

Downloaded image 1/45
Downloaded image 2/45
Downloaded image 3/45
Downloaded image 4/45
Downloaded image 5/45
Downloaded image 6/45
Downloaded image 7/45
Downloaded image 8/45
Downloaded image 9/45
Downloaded image 10/45
Downloaded image 11/45
Downloaded image 12/45
Downloaded image 13/45
Downloaded image 14/45
Downloaded image 15/45
Downloaded image 16/45
Downloaded image 17/45
Downloaded image 18/45
Downloaded image 19/45
Downloaded image 20/45
Downloaded image 21/45
Downloaded image 22/45
Downloaded image 23/45
Downloaded image 24/45
Downloaded image 25/45
Downloaded image 26/45
Downloaded image 27/45
Downloaded image 28/45
Downloaded image 29/45
Downloaded image 30/45
Downloaded image 31/45
Downloaded image 32/45
Downloaded image 33/45
Downloaded image 34/45
Downloaded image 35/45
Downloaded image 36/45
Downloaded image 37/45
Downloaded image 38/45
Downloaded image 39/45
Downloaded image 40/45
Downloaded image 41/45
Downloaded image 42/45
Downloaded image 43/45
Downloaded image 44/

You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [6]:
# m.layers[-1].visible = False
# m.add_raster(image, layer_name="Image")
# m

Map(bottom=37796022.0, center=[-22.17635, -51.25385], controls=(ZoomControl(options=['position', 'zoom_in_text…

## Initialize SAM class

Set `automatic=False` to enable the `SAM2ImagePredictor`.

In [19]:
sam = SamGeo2(
    model_id="sam2-hiera-large",
    automatic=False,
)

Specify the image to segment.

In [20]:
sam.set_image(image)

Display the map. Use the drawing tools to draw some rectangles around the features you want to extract, such as trees, buildings.

In [8]:
m

Map(center=[-22.17615, -51.253043], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Create bounding boxes

If no rectangles are drawn, the default bounding boxes will be used as follows:

In [27]:
import geopandas
from shapely.geometry import shape, box

if m.user_rois is not None:
    # m.user_rois is a GeoJSON dictionary. Convert it to a list of numerical bounding boxes.
    features = m.user_rois.get("features", [])
    valid_boxes = []
    for feature in features:
        geom_dict = feature.get("geometry")
        if geom_dict:
            try:
                # Create a shapely geometry object from the dictionary
                geom = shape(geom_dict)
                # Ensure it's a valid geometry and extract its bounds
                if geom.is_valid and not geom.is_empty:
                    bounds = geom.bounds  # (minx, miny, maxx, maxy)
                    valid_boxes.append(list(bounds))
            except Exception as e:
                # Handle cases where geometry might be malformed
                print(f"Skipping malformed or invalid geometry: {e}")
    if len(valid_boxes) > 0:
        boxes = valid_boxes
    else:
        # Fallback if user_rois was problematic or empty
        print("No valid user ROIs found, using default bounding boxes.")
        boxes = [
            [-51.2546, -22.1771, -51.2541, -22.1767],
            [-51.2538, -22.1764, -51.2535, -22.1761],
        ]
else:
    # If m.user_rois is None, use default bounding boxes
    boxes = [
        [-51.2546, -22.1771, -51.2541, -22.1767],
        [-51.2538, -22.1764, -51.2535, -22.1761],
    ]

NameError: name 'results' is not defined

## Segment the image

Use the `predict()` method to segment the image with specified bounding boxes. The `boxes` parameter accepts a list of bounding box coordinates in the format of [[left, bottom, right, top], [left, bottom, right, top], ...], a GeoJSON dictionary, or a file path to a GeoJSON file.

In [25]:
# sam.predict(boxes=boxes, point_crs="EPSG:4326", output="mask.tif", dtype="uint8")

No valid pixel coordinates found.


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

## Display the result

Add the segmented image to the map.

In [12]:
# m.add_raster("mask.tif", cmap="viridis", nodata=0, layer_name="Mask")
# m

Map(bottom=9449316.0, center=[-22.17635, -51.25385], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Use an existing vector dataset as box prompts

You can also use an existing vector dataset as box prompts. The following example uses an existing dataset of tree bounding boxes from GitHub.

In [21]:
geojson = (
    "/content/jj.geojson"
)

In [22]:
!pip install fiona

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 122.9 MB/s eta 0:00:00


Display the bounding boxes on the map.

In [20]:
# m = leafmap.Map()
# m.add_raster(image, layer_name="image")
# style = {
#     "color": "#ffff00",
#     "weight": 2,
#     "fillColor": "#7c4185",
#     "fillOpacity": 0,
# }
# m.add_vector(
#     geojson,
#     style=style,
#     zoom_to_layer=True,
#     layer_name="Bounding boxes",
#     info_mode=None,
# )
# m

## Segment trees with box prompts

Segment trees using the bounding boxes from the vector dataset.

In [12]:
boxes = [
    [115.62,126.57,185.05,201.42],
    [183.6,0,222,39.025],
    [74.429,56.775,109.82,89.242],
    [219.98,17.287,312.83,140.84],
    [117.5,46.247,180.43,111.08],
    [239.23,144.89,297.53,194.64],
    [19.093,179.55,105.66,239.2],
    [16.61,81.996,66.487,129.16],
    [82.253,1.4146,127.21,51.112],
    [127.87,7.1138,154.45,36.109],
    [225.79,178.29,298.42,238.4],
    [201.89,82.92,228.82,110.42],
    [341.7,194.46,359.95,237.82],
    [4.9002,26.11,53.403,69.359],
    [73.203,89.749,114.12,126.33],
    [0,192.1,17.477,238.85],
    [0.38534,0,61.952,36.19],
    [151.8,0.069454,189.34,40.008],
    [0,43.928,18.699,95.294],
    [175.18,127.22,202.14,155.32],
    ]

In [24]:
import json

with open("/content/jj.geojson") as f:
    gj = json.load(f)

features = gj["features"]


In [25]:
output_masks = "mask2.tif"
sam.predict(boxes=features, point_crs="EPSG:4326", output=output_masks, dtype="uint8")

ValueError: not enough values to unpack (expected 4, got 1)

Display the segmented masks on the map.

In [18]:
# m.add_raster(output_masks, nodata=0, opacity=0.5, layer_name="Tree masks")

## Post-processing

You can use the `region_groups()` method to clean up the segmentation results, such as removing small regions, and filling holes. In addition, you can compute geometric properties of the regions, such as area, perimeter, eccentricity, and solidity.

In [17]:
out_image = "tree_masks.tif"
out_vector = "tree_vector.geojson"
array, gdf = sam.region_groups(
    output_masks, min_size=200, out_vector=out_vector, out_image=out_image
)

RasterioIOError: mask2.tif: No such file or directory

In [20]:
# gdf.head()

,geometry,label,area,area_bbox,area_convex,area_filled,axis_major_length,axis_minor_length,eccentricity,equivalent_diameter_area,extent,orientation,perimeter,solidity,elongation
16,"POLYGON ((-5705817.622 -2532549.081, -5705817....",1,14067.0,24675.0,15275.0,14067.0,207.472166,90.980456,0.898722,133.830716,0.570091,0.469452,547.788889,0.920917,2.280404
3,"POLYGON ((-5705739.693 -2532549.081, -5705739....",2,2301.0,3599.0,2569.0,2301.0,74.881422,40.206720,0.843621,54.126927,0.639344,-0.708520,225.237590,0.895679,1.862411
1,"POLYGON ((-5705660.868 -2532549.081, -5705660....",3,3214.0,3816.0,3309.0,3214.0,111.893262,37.379812,0.942549,63.970242,0.842243,1.543199,257.982756,0.971290,2.993414
0,"POLYGON ((-5705576.967 -2532549.081, -5705576....",4,1852.0,2448.0,1909.0,1852.0,68.428893,36.633485,0.844630,48.559650,0.756536,1.517758,180.568542,0.970141,1.867933
18,"POLYGON ((-5705527.104 -2532549.081, -5705527....",5,27906.0,44128.0,32361.0,27906.0,215.450425,184.921852,0.513142,188.496745,0.632388,1.192862,1001.174711,0.862334,1.165089


## Display the cleaned masks

In [21]:
# m = leafmap.Map()
# m.add_raster(image, layer_name="Image")
# style = {
#     "color": "#ffff00",
#     "weight": 2,
#     "fillColor": "#7c4185",
#     "fillOpacity": 0,
# }
# m.add_raster(
#     out_image, colormap="tab20", nodata=0, opacity=0.7, layer_name="Tree masks"
# )
# m.add_vector(out_vector, style=style, zoom_to_layer=True, layer_name="Tree vector")
# m.add_vector(
#     geojson,
#     style={"color": "blue", "fillOpacity": 0},
#     layer_name="Bounding boxes",
#     info_mode=None,
# )
# m.add_layer_manager()
# m

Map(center=[-22.17635, -51.25385], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

![image](https://github.com/user-attachments/assets/b789a0e6-6e76-4b10-a9b8-3fc14676481f)

## Create a split map

In [22]:
# m = leafmap.Map()
# m.add_raster(image, layer_name="Image")
# m.split_map(
#     out_image,
#     image,
#     left_label="Tree masks",
#     right_label="Aerial imagery",
#     left_args={"colormap": "tab20", "nodata": 0, "opacity": 0.7},
# )
# m

Map(center=[-22.17635, -51.25385], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

![demo](https://github.com/user-attachments/assets/7bb0a65c-94f1-4cb6-9361-e79b47ec1e0a)

In [24]:
import json
from shapely.geometry import shape, mapping
import shapely.ops # Import shapely.ops
from pyproj import Transformer

# Load GeoJSON
with open("/content/tree_vector.geojson") as f:
    data = json.load(f)

# Create transformer (from source CRS → 4326)
# Change "EPSG:xxxx" to your original CRS
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

def transform_feature(feature):
    geom = shape(feature["geometry"])
    transformed_geom = shapely.ops.transform(transformer.transform, geom)
    feature["geometry"] = mapping(transformed_geom)
    return feature

data["features"] = [transform_feature(f) for f in data["features"]]

# Save output
with open("output_4326.geojson", "w") as f:
    json.dump(data, f)

print("\u2705 Converted to EPSG:4326 successfully!")

✅ Converted to EPSG:4326 successfully!
